# **数据规整：聚合、合并和重塑**
---

## **层次化索引**
---

In [ ]:
import pandas as pd
import numpy as np
data = pd.Series(np.random.randn(9),
                        index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]]
                        )

data

> **内层都可以索引**

In [ ]:
data.loc[['b', 'd'],3]

> **data.unstack()逆透视**

In [ ]:
data.unstack()

> **data.stack()透视**

In [ ]:
data.unstack().stack()

> **DataFrame每条轴都可以有分层索引**

In [ ]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                    ['Green', 'Red', 'Green']]
                    )
frame

In [ ]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
# frame.index
# frame.columns
# 注意索引名、列名、行名、字段名/列名/列标签是不一样的东西
frame

In [ ]:
frame['Ohio']

### **重排与分级排序**

> **df.swaplevel(x,y)交换排序位置**

In [ ]:
frame.swaplevel('key1', 'key2')

In [ ]:
frame.sort_index(level=1)

> **sort_index(level=0级别）**

In [ ]:
frame.swaplevel(0, 1).sort_index(level=0)

### **根据级别汇总统计**

In [ ]:
frame.sum(level='key2')

In [ ]:
frame

In [ ]:
frame.sum(level='color', axis=1)

### **使用DataFrame的列进行索引**

> **set_index()**

In [ ]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two',
                     'two', 'two'],
                     'd': [0, 1, 2, 0, 1, 2, 3]})
frame

In [ ]:
frame2 = frame.set_index(['c','d'],drop=False) # 保留相关列
frame2

## **合并数据集**  
---
> - pandas.merge可根据一个或多个键将不同DataFrame中的行连接起来。SQL或其他关系型数据库的用户对此应该会比较熟悉，因为它实现的就是数据库的join操作。  
> - pandas.concat可以沿着一条轴将多个对象堆叠到一起。  
> - 实例方法combine_first可以将重复数据拼接在一起，用一个对象中的值填充另一个对象中的缺失值。

### **类似数据库的合并**

> **pd.merge(df1,df2,on='key')**

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df1

In [ ]:
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df2

In [ ]:
pd.merge(df1, df2)

In [ ]:
pd.merge(df1, df2, on='key')

In [ ]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                     'data1': range(7)})
df3

In [ ]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                        'data2': range(3)})
df4

In [ ]:
pd.merge(df3,df4,left_on='lkey',right_on='rkey')

In [ ]:
pd.merge(df1, df2, how='outer')

> **连接方式**  
> **inner ：内连接**  
> **left : 左连接**  
> **right :右连接**  
> **Outer : 外连接**

In [ ]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df1

In [ ]:
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                     'data2': range(5)})
df2

In [ ]:
pd.merge(df1,df2,on='key',how='left')

> **多键合并on=['key1', 'key2']传入一个由列名组成的列表即可**

In [ ]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                    'key2': ['one', 'two', 'one'],
                    'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                         'key2': ['one', 'one', 'one', 'two'],
                          'rval': [4, 5, 6, 7]})

pd.merge(left, right, on=['key1', 'key2'], how='outer')

In [ ]:
pd.merge(left, right, on='key1')

> **suffixes('_left', '_right')指定重复项的列明**

In [ ]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

> merge参数
![](img)

### **索引上的合并**

In [ ]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                     'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
pd.merge(left1, right1, left_on='key', right_index=True) # 右边使用索引作为合并的字段

> **merge默认取交集，可加how='outer'取并集**

In [ ]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

> **多层次的索引，多键合并**

In [ ]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                                'Nevada', 'Nevada'],
                        'key2': [2000, 2001, 2002, 2001, 2002],
                         'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                        index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                               'Ohio', 'Ohio'],
                               [2001, 2000, 2000, 2000, 2001, 2002]],
                       columns=['event1', 'event2'])

pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

In [ ]:
pd.merge(lefth, righth, left_on=['key1', 'key2'],
          right_index=True, how='outer')

> **直接使用索引轴合并**

In [ ]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                        index=['a', 'c', 'e'],
                       columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                     index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

### **Join方法**

In [ ]:
left2.join(right2, how='outer')

### **concat()轴向连接**

In [ ]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
pd.concat([s1,s2,s3])

In [ ]:
pd.concat([s1,s2,s3],axis=1)

In [ ]:
s4 = pd.concat([s1, s3])
s4

In [ ]:
pd.concat([s1, s4], axis=1)

> **Join='inner'取交集**

In [ ]:
pd.concat([s1, s4], axis=1, join='inner')

In [ ]:
pd.concat([s1, s4], axis=1, join_axes=['a', 'c', 'b', 'e'])

> **合并中还可以创建层次化索引，使用keys**

In [ ]:
result = pd.concat([s1, s1, s3], keys=['one','two', 'three'])
result

In [ ]:
result.unstack() # 逆透视一下

In [ ]:
pd.concat([s1, s2, s3], axis=1, keys=['one','two', 'three'])

### **同样的逻辑也适用于DataFrame对象**

In [ ]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                    columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                     columns=['three', 'four'])
df1                     

In [ ]:
df2

In [ ]:
pd.concat([df1,df2],axis=1,keys=['level1', 'level2'])

In [ ]:
# 如果传入的值的字典，那字典的键会被当作keys
pd.concat({'level1': df1, 'level2': df2}, axis=1) 

In [ ]:
# 设置索引层次水平
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
       names=['upper', 'lower'])

> ![](img/concat%E5%8F%82%E6%95%B0.jpg)

### **合并重叠数据combine_first打补丁**

In [ ]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
               index=['f', 'e', 'd', 'c', 'b', 'a'])
a

In [ ]:
import numpy as np
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
b

In [ ]:
b[:-2].combine_first(a[2:])

## **重塑和轴向旋转**
---

### **重塑层次化索引**
> **df.stack()透视和df.unstack()逆透视**

In [ ]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                      index=pd.Index(['Ohio','Colorado'], name='state'),
                     columns=pd.Index(['one', 'two', 'three'],
                       name='number'))
data

In [ ]:
result = data.stack()
result

In [ ]:
result.unstack()

###  **将“长格式”旋转为“宽格式”**

In [ ]:
data = pd.read_csv('examples/macrodata.csv')
data.head()

In [ ]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns={0: 'value'})

In [ ]:
data # date和item通常就是主键

In [ ]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

In [ ]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]


In [ ]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

> **_注意，pivot其实就是用set_index创建层次化索引，再用unstack重塑：_**

### **将“宽格式”旋转为“长格式”**

In [ ]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                       'A': [1, 2, 3],
                       'B': [4, 5, 6],
                      'C': [7, 8, 9]})
df

In [ ]:
melted = pd.melt(df, ['key'])
melted

In [ ]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

In [ ]:
reshaped.reset_index() 